# STOR 609 Assignment 1 

In order to get an understanding of the problems in this assessment I initially wrote separate functions to solve both problems in $\texttt{R}$. These functions were later translated to, and generalised in, Python.

My aim in this notebook is to give an account of most of the second and third items of the assessment brief in section $9.3$ using my initial code to both problems. In particular, the first bullet point and most of the third and fourth bullet points for both problems.
- Brief explanation of how (each) problem can be solved with backtracking.
- Source code for backtracking operations
- Concrete example of using code to solve the problem


# Problem 1

## Explanation of backtracking for partitions.

I considered a partition as an ordered list of integers that sum to a given number $n$. For example if $n=10$, then $[6,3,1]$ is a partition, where I put the numbers in descending order. I also used an order between partitions. For example $[6,3,1]$ comes before $[6,2,2]$, which comes before $[6,2,1,1]$. This is the reverse of dictionary (lexicographic) order, where smaller numbers would normally come first.

The largest possible number in a partition of $n$ is $n$ itself. The singleton list $n$ is the root.

The backtracking algorithm works by completing a partial partition by suggesting the smallest current number in the list, and if this is too big, reducing it by 1 until the smallest number in the list is 1, at which point we move to the smallest number which is greater than one and reduce this by 1. The process terminates when all numbers are $1$. 

For example, given the partially complete partition $[6,3]$ of $10$, we would first suggest $[6,3,3]$ then $[6,3,2]$ then obtain $[6,3,1]$. We check the sum at each iteration and accept at the end of this chain. Once the last number in the list is $1$, we $\textit{backtrack}$ to $[6,3]$, and reduce the last entry by one. The next suggested partial partition would be $[6,2]$, as we have exhausted possibilities with $3$ as the smallest number greater than $1$, and so the process continues to suggest $[6,2,2]$ then obtain $[6,2,1]$. 

By maintaining the order we ensure that we do not miss a partition.

## R implementation - partition()

I tried to keep the format of the code the same as that of the pseudocode on Wikipedia, to enable later generalisation. Below is the function I wrote and some example output.

In [1]:
partition <- function(n, partial_partition){
####################################################
##  Function to print the partitions of a number  ##
##  using recursive backtracking.                 ##
##  Inputs: integer n                             ##
##          list of integers partial_partition    ##
##  Output: prints the partitions of n.           ##
####################################################
    # Reject 
    if( (sum(partial_partition) > n)){
        return()
    }
    if( length(partial_partition)==0 | is.null(partial_partition)){
        s <<- NULL
        return()
    }
    
    # Accept
    if(sum(partial_partition) == n){
        print(partial_partition)
    }
    
    # First
    s <<- partial_partition
    if( sum(partial_partition) < n ){
        s <<- c(partial_partition, partial_partition[length(partial_partition)])
    } else {
        s <<- s[which(s!=1)]
        s[length(s)] <<- s[length(s)] - 1
    }
    
    while( !is.null(s) && length(s)>0 ){
        # Recursive step
        partition(n,s)
        
        # Next
        if( is.null(s) | length(s)==0){
            break
        }
        if( sum(s == 1) == n){
            s <<- NULL
            break
        } else {
            s <<- s[which(s!=1)]
            s[length(s)] <<- s[length(s)] - 1
        }
    }
}

### Examples

In [2]:
partition(5, c(5))

[1] 5
[1] 4 1
[1] 3 2
[1] 3 1 1
[1] 2 2 1
[1] 2 1 1 1
[1] 1 1 1 1 1


In [3]:
partition(10,c(10)) |> capture.output() |> length()

[1] 42

# Problem 2
## Explanation of backtracking for Gray codes.

A gray code is a sequence of binary codewords of length $n$ where each subsequent codeword differs from the previous one by only one bit, and the last codeword is one bit away from the starting codeword. As the codewords are binary there are two choices for each bit, and all the choices multiply so the code is of length $2^n$.

For example consider $n=2$. Here a gray code is a sequence of length $2^2=4$: 

$$00\to 01 \to 11 \to 10.$$

To get the next codeword in a partial sequence one can try to always flip the rightmost bit, and check if this new codeword is already in the sequence. For example,

$$00 \to 01 \to 00 \ \times$$

If the new codeword is already in the sequence, we can $\textit{backtrack}$ to the previous word and try next to flip the bit one place to the left of the previous attempt. Here, that gives the partial sequence:

$$00 \to 01 \to 11 \checkmark$$

We may need to do this multiple times working from right to left. For my implementation I store the position of the bit which was last changed, so that when backtracking one can decrement the value of this position to find the next bit location. 

## R implementation - gray_code()

Again I kept to the same general format as on Wikipedia. However in the accept section I introduced an early return clause. 

In [4]:
gray_code <- function(n, part_code){
#######################################################
##  Function to print a binary gray code of a given  ##
##  codeword length using recursive backtracking.    ##
##  Inputs: integer n                                ##
##          list of lists of integers part_code      ##
##          (codewords are lists of integers)        ##
##  Output: prints a gray code.                      ##
##  Example: for a gray code of codeword length 3    ##
##           initial codeword 000, call              ##
##           gray(3, list(rep(0,3)))                 ##
#######################################################
  # Reject
  if( length(unique(part_code)) < length(part_code)){
    return()
  }
    
  # Accept
  # if the correct length and only 1 bit away from the start (circular property)
  if(length(part_code) == 2^n && sum(part_code[[length(part_code)]]) == 1){
    print(part_code)
    return()
  }
  
  # First
  s <<- part_code
  #000 is encountered initially
  if( sum(s[[length(s)]] == rep(0,n)) == n && length(s) == 1){
    # start at least sig. bit - a pointer to which digit should flip next
    s_index <<- n
    s_index <<- c(s_index)
    # change last bit of first codeword e.g. 000 to 001
    last_codeword <- s[[length(s)]] 
    last_codeword[s_index[length(s_index)]] <- (last_codeword[s_index[length(s_index)]] + 1)%%2
    s <<- c(s, list(last_codeword))
  } else{
    s_index <<- c(s_index,n)
    last_codeword <- s[[length(s)]]
    last_codeword[s_index[length(s_index)]] <- (last_codeword[s_index[length(s_index)]] + 1)%%2
    s <<- c(s, list(last_codeword))
  }
  
  while( length(s) < 2^n + 1){
    gray_code(n,s)
    
    # Next
    if(length(s) == 2^n && sum(s[[length(s)]]) == 1){
      break
    }
    else if( s_index[length(s_index)] == 1){
      # remove the last entry (1) and decrement previous value
      s_index <<- s_index[-length(s_index)]
      previous_index <- s_index[length(s_index)]
      previous_index <- previous_index - 1
      s_index <<- s_index[-length(s_index)]
      s_index <<- c(s_index, previous_index)
      s <<- s[-length(s)]
    } else {
      # decrement last value
      last_index <- s_index[length(s_index)]
      last_index <- last_index - 1
      s_index <<- s_index[-length(s_index)]
      s_index <<- c(s_index, last_index)
      # set last codeword of s to be penultimate one with flipped bit at s_index
      # first remove the last code word
      s <<- s[-length(s)]
      last_codeword <- s[[length(s)]]
      last_codeword[s_index[length(s_index)]] <- (last_codeword[s_index[length(s_index)]] + 1)%%2
      s <<- c(s, list(last_codeword))
    }
  }
}

In [5]:
gray_code(3,list(rep(0,3))) 

[[1]]
[1] 0 0 0

[[2]]
[1] 0 0 1

[[3]]
[1] 0 1 1

[[4]]
[1] 0 1 0

[[5]]
[1] 1 1 0

[[6]]
[1] 1 1 1

[[7]]
[1] 1 0 1

[[8]]
[1] 1 0 0



This output is a printout of the list of lists, with each list item printed over $3$ lines. To verify the output length we can divide the captured print output by $3$ and check the correct code length ($=2^n$).

In [6]:
gray_code(10,list(rep(0,10))) |> capture.output() |> length()/3

[1] 1024